In [1]:
import pyspark

In [8]:
from pyspark.sql import *

spark = SparkSession.builder\
    .appName("StructuredNetworkWordCount")\
    .getOrCreate()

In [13]:
df = spark.read.format("csv").load("../docker/airflow/data/processed/gtfs/transilien-gtfs-2022-06-24.csv", header=True)

In [65]:
from pyspark.sql.types import *

relation_schema = StructType()\
    .add("Gare", StringType(), True)\
    .add("Relation", IntegerType(), True)

df_relation = spark.read.format("csv")\
    .options(delimiter=';')\
    .schema(relation_schema)\
    .load("../docker/airflow/data/reference/relation_ordre_RER_B.csv", header=False)

In [66]:
df_relation.show(truncate=False)

+-----------------------------------------+--------+
|Gare                                     |Relation|
+-----------------------------------------+--------+
|Aéroport Charles de Gaulle 2 (Terminal 2)|1       |
|Aéroport CDG 1 (Terminal 3) - RER        |2       |
|Parc des Expositions                     |3       |
|Villepinte                               |4       |
|Sevran Beaudottes                        |5       |
|Mitry - Claye                            |6       |
|Villeparisis - Mitry-le-Neuf             |7       |
|Vert Galant                              |8       |
|Sevran - Livry                           |9       |
|Aulnay-sous-Bois                         |10      |
|Le Blanc-Mesnil                          |11      |
|Drancy                                   |12      |
|Le Bourget                               |13      |
|La Courneuve - Aubervilliers             |14      |
|La Plaine Stade de France                |15      |
|Gare du Nord                             |16 

In [71]:
df = df.drop("_c0")

In [72]:
df.show(500, truncate=False)

+-------------------------------------------------+-----------+-------------+---------------+-----------------------------------------+--------------+---------------------------------+-----------+--------------+-----------------------------------------+-----------+------------+--------------+---------+
|trip_id                                          |trip_line  |trip_headsign|trip_short_name|destination                              |destination_id|origin                           |origin_id  |time_departure|arrival                                  |arrival_id |time_arrival|time_travelled|direction|
+-------------------------------------------------+-----------+-------------+---------------+-----------------------------------------+--------------+---------------------------------+-----------+--------------+-----------------------------------------+-----------+------------+--------------+---------+
|IDFM:TN:SNCF:7057d5db-4167-45fb-80fd-acde4637e487|IDFM:C01743|IPEL         |IPOI01     

In [43]:
from pyspark.sql.functions import *

# Direction au nord

val = ['IDFM:73596', 'IDFM:73699', 'IDFM:73568', 'IDFM:73547', 'IDFM:73491', 'IDFM:72646', 'IDFM:72648', 'IDFM:72652', 'IDFM:72641', 'IDFM:72598', 'IDFM:72211', 'IDFM:71410', 'IDFM:68916', 'IDFM:69065', 'IDFM:73482', 'IDFM:73439']
df.groupBy("origin", "origin_id", "arrival", "direction")\
    .agg({'time_travelled': 'avg'})\
    .filter(df.direction == 1)\
    .filter(df.origin_id.isin(val))\
    .show(100)

+--------------------+----------+--------------------+---------+-------------------+
|              origin| origin_id|             arrival|direction|avg(time_travelled)|
+--------------------+----------+--------------------+---------+-------------------+
|        Gare du Nord|IDFM:71410|La Plaine Stade d...|      1.0| 262.22222222222223|
|        Gare du Nord|IDFM:71410|Aéroport CDG 1 (T...|      1.0| 1587.7551020408164|
|Villeparisis - Mi...|IDFM:68916|       Mitry - Claye|      1.0|              240.0|
|          Villepinte|IDFM:73547|Parc des Expositions|      1.0|  90.33898305084746|
|     Le Blanc-Mesnil|IDFM:72648|    Aulnay-sous-Bois|      1.0| 105.04273504273505|
|Aéroport CDG 1 (T...|IDFM:73596|Aéroport Charles ...|      1.0| 145.27777777777777|
|              Drancy|IDFM:72652|     Le Blanc-Mesnil|      1.0|              100.0|
|Parc des Expositions|IDFM:73568|Aéroport CDG 1 (T...|      1.0|  291.0169491525424|
|         Vert Galant|IDFM:73482|Villeparisis - Mi...|      1.0| 

In [73]:
# Direction du Sud

df_sud = df.groupBy("origin", "origin_id", "arrival", "direction") \
    .agg({'time_travelled': 'avg'})\
    .filter(df.direction == 0)\
    .filter(df.origin_id.isin(val))\
    .join(df_relation, df.origin == df_relation.Gare, "inner")\
    .orderBy("Relation")

In [84]:
df_sud.filter("Relation >= 3 and Relation < 15").show()

+--------------------+----------+--------------------+---------+-------------------+--------------------+--------+
|              origin| origin_id|             arrival|direction|avg(time_travelled)|                Gare|Relation|
+--------------------+----------+--------------------+---------+-------------------+--------------------+--------+
|Parc des Expositions|IDFM:73568|          Villepinte|      0.0|  90.17543859649123|Parc des Expositions|       3|
|          Villepinte|IDFM:73547|   Sevran Beaudottes|      0.0| 112.45614035087719|          Villepinte|       4|
|   Sevran Beaudottes|IDFM:73491|    Aulnay-sous-Bois|      0.0| 168.24561403508773|   Sevran Beaudottes|       5|
|       Mitry - Claye|IDFM:69065|Villeparisis - Mi...|      0.0| 210.51724137931035|       Mitry - Claye|       6|
|Villeparisis - Mi...|IDFM:68916|         Vert Galant|      0.0|              150.0|Villeparisis - Mi...|       7|
|         Vert Galant|IDFM:73482|      Sevran - Livry|      0.0|              21

In [85]:
df_sud.filter("Relation >= 3 and Relation < 15").agg({"avg(time_travelled)": "sum"}).show()

+------------------------+
|sum(avg(time_travelled))|
+------------------------+
|      1741.9161734922013|
+------------------------+

